## Week 6-2: Simulation to detect campaign finance payments

This notebook replicates the analysis in the post [Statistical Model Strongly Suggests the Stormy Daniels Payoff Came from the Trump Campaign](https://medium.com/@whstancil/statistical-model-strongly-suggests-the-stormy-daniels-payoff-came-from-the-trump-campaign-7c09c300cb18) and analyzes a robustness problem in the argument: the probability of finding five payments that add up to something close to $130,000 increases quickly as the number of payments to choose from increases, and the authors arbitrarily chose 10.

A deeper problem is that this statistical test has all sorts of underlying assumptions about how the data relates to reality. [For example](https://twitter.com/pbump/status/972216289384157188), why would the campaign choose to report an illegal payment in their public data? However, even on the test's own terms, the sensitivity on the choice of 10 makes the results look a lot less robust.

From the original post:

 >As is well-known by now, in late October 2016, Donald Trump’s personal attorney Michael Cohen paid adult star Stormy Daniels \$130,000 in order to purchase her silence about an alleged affair a decade earlier. The exact set of facts around this payment remains shrouded in mystery, with Cohen maintaining that he made the payment alone, without Trump’s knowledge, and with personal funds ... However, sharp-eyed observers have noted that, in late October 2016, the Trump campaign made a series of five large payments to Trump-affiliated entities, totaling \$129,999.72. 

The core of the argument is:

> Ultimately, our model suggests that the probability of a set of payments coincidentally coming so close to \$130,000 is approximately 0.1%, or one out of one thousand. In other words, about 99.9% of the time, random chance would not produce a set of payments this close to \$130,000. Therefore, the probability that the Trump campaign payments were related to the Daniels payoff is very high.

Their model calculates the probability of getting this close if the last few payments were truly random, by repeatedly simulating a set of 10 campaign payments and seeing how close you can get to \$130,000 by adding up some set of them. You can't get very close very often. However, there's no reason to choose to look at only the last 10 payments. If you look at the last 20 payments instead, the probability of finding a subset that some to within a dollar of \$130,000 rises to 40%!

The data is "a list of all Trump campaign payments to Trump-affiliated entities" compiled from FEC data by DC attorney and blogger Susan Simpson, available [here](from https://twitter.com/TheViewFromLL2/status/971771806414725130).

### Things to think about

Of course the issue of whether there was an illegal payoff here has since been overtaken by Cohen's guilty plea, but working through this problem touches on a number of deep-ish statistical ideas.

1) This type of "the alternative is super unlikely" statistical argument appears in many places. It's the core idea of significance testing and p-values. See [this chapter](https://towcenter.gitbooks.io/curious-journalist-s-guide-to-data/content/analysis/arguing_from_the_odds.html) of *The Curious Journalist's Guide to Data.* 

2) But again, how does this data relate to reality? What are the FEC rules and typical campaign practice for what is reported and when? When politicians have pulled shady stuff in the past, how did it look in the data? We desperately need domain knowledge here. For an example of what that looks like, see [Fivethirtyeight's critique of statistical tests for tennis fixing](https://fivethirtyeight.com/features/why-betting-data-alone-cant-identify-match-fixers-in-tennis/).

3) What is the correct number of payments to use as a baseline for simulation? In other words, what is our universe of events that were using to calculate the probability of this particular thing happening? The authors [note](https://medium.com/@whstancil/hi-david-its-not-fixed-at-5-that-s-just-how-many-payments-made-up-the-original-discovery-9c3696b7f694) "the set was fixed at ten because we’re trying to estimate the odds of the original discovery, which was found in a series of eight or so payments" which has a pleasant "let's let the data dictate the model" kind of rationale. But the whole concept of frequentist inference is that we reason about the statistics of the process, independent of observed data, so it's not clear to me if this argument makes sense. Or that any argument about the "right" or "objective" number of payments to check with this method can really be solid. I'd prefer to see a fully Bayesian attempt, modeling the generation process for the entire observed payment stream with and without the Stormy payoff.

4) At the highest level of generality, the problem of arbitrarily chosen parameters is a red flag in any analysis. Whenever you see a chooseable constant, ask how different the answer would be if a different constant was chosen.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Load and clean data

In [29]:
df = pd.read_csv('data/15-16-cycle-disbursements-to-trump-properties.csv')
df.head()

,disbursement_date,recipient_name,recipient_state,disbursement_amount,disbursement_description,recipient_city,disbursement_purpose_category,pdf_url,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,4/30/15 0:00,TRUMP INTERNATIONAL HOTEL AND TOWER,NY,"$1,380.54",TTW - TRAVEL: LODGING,NEW YORK,TRAVEL,http://docquery.fec.gov/cgi-bin/fecimg/?201507...,NaN,NaN,NaN,NaN,NaN
1,5/5/15 0:00,TRUMP TOWER COMMERCIAL LLC,NY,"$9,583.33",TTW - RENT,HICKSVILLE,ADMINISTRATIVE,http://docquery.fec.gov/cgi-bin/fecimg/?201507...,NaN,NaN,NaN,NaN,NaN
2,6/16/15 0:00,THE TRUMP CORPORATION,NY,"$37,993.04","TTW - ONE-TIME REIMBURSEMENT FOR FACILITY, RE...",NEW YORK,OTHER,http://docquery.fec.gov/cgi-bin/fecimg/?201507...,NaN,NaN,NaN,NaN,NaN
3,6/16/15 0:00,TRUMP SOHO,NY,"$3,240.96",TTW - TRAVEL: LODGING,NEW YORK,TRAVEL,http://docquery.fec.gov/cgi-bin/fecimg/?201507...,NaN,NaN,NaN,NaN,NaN
4,6/16/15 0:00,TRUMP TOWER COMMERCIAL LLC,NY,"$9,583.33",TTW - RENT,HICKSVILLE,ADMINISTRATIVE,http://docquery.fec.gov/cgi-bin/fecimg/?201507...,NaN,NaN,NaN,NaN,NaN


In [30]:
# Following the original post above, drop all payments labelled as RENT (but not RENTAL)
# Also, drop everything later than October 26, when Cohen was notified the funds arrived 
# (see https://twitter.com/TheViewFromLL2/status/971771806414725130)
payments = df.disbursement_amount[
    ~(df.disbursement_description.str.contains('RENT') & ~df.disbursement_description.str.contains('RENTAL')) & 
    (pd.to_datetime(df.disbursement_date) < '10/27/2016')]
len(payments)

127

In [31]:
# Convert the payments column to numbers (right now it's a string) by dropping dollar signs and commas 
payments = payments.str.replace('$','').str.replace(',','').astype(float)

In [32]:
# Following the original, drop any payment below $1000
payments = payments[payments >= 1000]

In [33]:
len(payments)

54

In [34]:
# these are the payments thst Simpson spotted
paylist = [194, 195, 196, 197, 200]
print(payments[paylist])
print(payments[paylist].sum())

194    13431.88
195    18731.90
196    79043.94
197     8544.00
200    10248.00
Name: disbursement_amount, dtype: float64
129999.72


### Function for finding closest matching subset of payments

To repeat the analysis, we need to know how close we can get to \$130,000 by summing any possible subset of a set of payments. So we'll write that function first. This version operates by exhaustive search of all possible subsets, so it's slow but accurate.

In [8]:
# Return subset of the numbers that match target most closely
def closest_match(numbers, target):
    num_numbers = len(numbers)
    num_subsets = int(math.pow(2,num_numbers))                        

    closest_miss = 1e10
    
    # enumerate all subsets, by counting up through the 2^n combinations
    for which_subset in range(0, num_subsets):
        subset = []
        idx = 0
        while idx<num_numbers:
            if int(which_subset) & (1 << idx):  # if this bit is set in binary representation,
                  subset.append(numbers[idx])   # add in the corresponding element
            idx+=1

        miss = abs(sum(subset)-target)
        if miss < closest_miss:
            closest_miss = miss
            closest_subset = subset
        
    return closest_subset


Sure enough, this finds the payments that Simpson marked as suspicious, when we run it on the last 10 transactions.

In [9]:
best_of_10 = closest_match(payments.tail(10).values, 130000)

In [10]:
best_of_10

[13431.879999999999, 18731.900000000001, 79043.940000000002, 8544.0, 10248.0]

In [11]:
sum(best_of_10)

129999.72

It finds an even closer match on last 20

In [12]:
best_of_20 = closest_match(payments.tail(20).values, 130000)

In [13]:
best_of_20

[7173.6000000000004,
 1127.03,
 1021.4299999999999,
 2172.4499999999998,
 1343.0,
 13431.879999999999,
 79043.940000000002,
 8544.0,
 16142.610000000001]

In [14]:
sum(best_of_20)

129999.94

### A faster search 
Searching 2^20 subsets takes several seconds, and we want to run this thousands of times to do our simulation over random sets of payments. So for performance reasons, we're going to modify this function so that it just finds the first subset within threshold of the target -- not the closest. This lets it bail very much quicker, in most cases, and it tells us all we need to know to evaluate any given set of payments: is there *any* subset that gets sufficiently close?

In [15]:
# Determine if there is a subset of the numbers that sum to within threshold of the target
# Returns sum within threshold if a matching subset is found, else False.
def threshold_match(numbers, target, threshold):
    num_numbers = len(numbers)
    num_subsets = int(math.pow(2,num_numbers))                        
    
    # enumerate all subsets, by counting up through the 2^n combinations
    for which_subset in range(0, num_subsets):
        sum = 0
        idx = 0
        while idx<num_numbers:
            if int(which_subset) & (1 << idx):  # if this bit is set in binary representation,
                  sum += numbers[idx]           # add in the corresponding element
            idx+=1

        # if we find a close sum, we are done early
        if abs(sum-target) <= threshold:
            return sum
        
    return False


This should find the identified payments again

In [16]:
threshold_match(payments.tail(10).values, 130000, 0.28)

129999.72

And just to prove it's working, there isn't any set of payments within 10 that comes closer

In [17]:
threshold_match(payments.tail(10).values, 130000, 0.27)

False

### The main probabilistic analysis

Now we're going to replicate the analysis in the post and look at transaction sets of different sizes. They analyzed 10 and 15 within the post, but we'll also look at 20 -- and see that getting close to \$130,000 is dramatically more likely than when we only look at 15 items.

In [25]:
# subset_size: how large a group of random campaign finance payments are we looking through?
# dollar_threshold: how close to $130,000 do we need to get to count it as a hit?
# num_samaples: increase this for more accurate estimation of the probability
def probability_of_closeness(subset_size, dollar_threshold, num_samples):
    # how many sets of payments that get within dollar_threshold have we found?
    hits = 0

    # generate num_samples different sets of payments by sampling from all payments
    for i in range(0, num_samples):
        # pick subset_size numbers randomly from payments
        subset = payments.sample(subset_size)

        # Is there a close match to $130,000?
        close_sum = threshold_match(subset.values, 130000, dollar_threshold)

        # Count "hits", printing out the close sums we find
        if close_sum:
            print(f'{i}: {close_sum}')
            hits += 1

    # print the fraction of hits
    return hits/num_samples

Armed with this function, we'll start by repeating the calculation with 10 payments, as the post does. We'll calculate the probability of getting within \$1 of \$130,000, taking 1000 samples for accuracy (as only 2 or 3 will get this close.)

In [19]:
p10 = probability_of_closeness(10, 1, 1000)

495: 129999.54000000001
642: 130000.54999999999


In [20]:
p10

0.002

We've replicated the results. Probability of getting within \$1 is 0.003, which agrees with the chart in the post (showing 0.001 for \$0.24 and 0.005 for \$1.44)

In [21]:
p15 = probability_of_closeness(15, 1, 1000)

13: 130000.61000000002
32: 130000.45999999999
37: 130000.55
41: 129999.93
54: 130000.97000000002
56: 129999.59
61: 129999.16
65: 129999.97
66: 130000.62000000001
67: 130000.14
68: 129999.56
70: 130000.48
73: 129999.18000000001
76: 130000.65000000001
79: 129999.24000000002
89: 130000.14
100: 130000.83
114: 129999.18000000002
131: 129999.87
150: 129999.48999999999
162: 130000.58
169: 130000.28
183: 130000.6
184: 129999.38
194: 129999.24
216: 130000.94
217: 129999.91
219: 130000.81000000001
220: 129999.11000000002
234: 129999.36
242: 129999.22
249: 130000.91
262: 129999.55000000002
264: 129999.72
273: 130000.38
277: 130000.62
282: 129999.99
286: 129999.37000000001
291: 129999.91
297: 130000.0
301: 130000.1
310: 129999.31999999999
311: 130000.86
314: 130000.24
330: 129999.98
335: 129999.01
336: 129999.9
344: 130000.81999999999
349: 129999.37
356: 129999.67000000001
360: 130000.40000000001
367: 130000.48000000001
377: 129999.52000000002
383: 129999.31000000001
393: 130000.83000000002
416: 1

In [22]:
p15

0.131

And now for the main event: how likely is it that we can get within \$1 of \$130,000 if we take 20 random payments? It turns out, pretty likely. 

1000 samples of size 20 takes about an hour to run, so we'll use only 100 this time. It's no problem here as hits are very common.

In [23]:
p20 = probability_of_closeness(20, 1, 100)

0: 129999.59000000001
3: 130000.86
5: 130000.45
6: 130000.74
7: 129999.71999999999
9: 129999.0
10: 129999.62
11: 130000.85
12: 129999.86
13: 130000.9
14: 130000.98000000001
16: 130000.67000000001
18: 129999.99
19: 130000.97999999998
20: 129999.36
21: 130000.17
22: 130000.43
24: 130000.33999999998
25: 129999.58000000002
26: 129999.06000000001
27: 129999.66999999998
28: 129999.96
29: 130000.18
30: 129999.95999999998
32: 130000.39
33: 130000.19
34: 130000.04000000001
35: 129999.82999999999
36: 129999.28
37: 130000.86000000002
39: 129999.82
40: 129999.76
41: 129999.10999999999
42: 130000.68
43: 130000.94
44: 130000.71
45: 130000.51999999999
46: 130000.70000000001
47: 130000.78
49: 129999.07
50: 130000.1
51: 130000.71
53: 129999.05999999998
54: 129999.04
56: 129999.52
57: 129999.61000000002
58: 130000.72
59: 130000.29
60: 130000.99
61: 129999.75
62: 130000.99
63: 130000.05
64: 130000.16999999998
65: 130001.0
66: 129999.97
67: 129999.75
68: 129999.98
70: 129999.99
71: 129999.45
72: 129999.06

In [24]:
p20

0.82

This is a major challenge for the argument in the post. Having a subset of the payments sum to within a dollar of \$130,000 is very unlikely if we only look at the last 10 payments (p=0.003, or 0.3%), but not that unlikely if we look at the last 15 (p=0.131, or 13.1%) and a safe bet if you look at the last 20 (p=0.82, or 82%). Why did the authors of the analysis choose 10 and not 15 or 20? The result of the analysis is very sensitive to this "magic" number.